In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
!mv iris.data iris.csv

--2021-09-06 06:18:42--  https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551 (4.4K) [application/x-httpd-php]
Saving to: ‘iris.data’

iris.data           100%[===================>]   4.44K  --.-KB/s    in 0s      

2021-09-06 06:18:43 (184 MB/s) - ‘iris.data’ saved [4551/4551]



In [5]:
BUCKET_NAME = 'hassan-ai-platform-temp'
!gsutil mb gs://$BUCKET_NAME
!gsutil cp ./iris.csv gs://$BUCKET_NAME

Creating gs://hassan-ai-platform-temp/...
ServiceException: 409 A Cloud Storage bucket named 'hassan-ai-platform-temp' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.
Copying file://./iris.csv [Content-Type=text/csv]...
/ [1 files][  4.4 KiB/  4.4 KiB]                                                
Operation completed over 1 objects/4.4 KiB.                                      


In [43]:
!mkdir ai_platform_training_custom_container
TRAINING_APP_FOLDER = 'ai_platform_training_custom_container'

In [9]:
%pwd
%cd ..


/home/jupyter


## Docker File

In [10]:
%%writefile ./ai_platform_training_custom_container/Dockerfile
FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U scikit-learn==0.20.4 pandas==0.24.2
WORKDIR /app
COPY train.py .
ENTRYPOINT ["python", "train.py"]

Overwriting ./ai_platform_training_custom_container/Dockerfile


In [11]:
PROJECT_ID='qp-acc-aia-migrate-2021-06'
IMAGE_NAME='ai_platform_training_custom_container_image'
IMAGE_TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, IMAGE_TAG)
!gcloud builds submit --tag $IMAGE_URI $TRAINING_APP_FOLDER

ERROR: (gcloud.builds.submit) argument --tag/-t: expected one argument
Usage: gcloud builds submit [[SOURCE] --no-source] [optional flags]
  optional flags may be  --async | --no-cache | --config | --disk-size |
                         --gcs-log-dir | --gcs-source-staging-dir | --help |
                         --ignore-file | --machine-type | --pack | --region |
                         --no-source | --substitutions | --suppress-logs |
                         --tag | --timeout | --worker-pool

For detailed information on this command and its flags, run:
  gcloud builds submit --help


In [12]:
import time
print(IMAGE_URI)
JOB_NAME = "Iris_data_custom_container_job"
JOB_DIR = 'gs://' + BUCKET_NAME + '/iris-job-custom-container-dir'
REGION = 'us-west1'
!gcloud ai-platform jobs submit training $JOB_NAME \
--region=$REGION \
--job-dir=$JOB_DIR \
--master-image-uri=$IMAGE_URI \
--scale-tier=BASIC

gcr.io/qp-acc-aia-migrate-2021-06/ai_platform_training_custom_container_image:latest
ERROR: (gcloud.ai-platform.jobs.submit.training) Resource in projects [qp-acc-aia-migrate-2021-06] is the subject of a conflict: Field: job.job_id Error: A job with this id already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A job with this id already exists.
    field: job.job_id


In [52]:
!gcloud ai-platform jobs describe Iris_data_custom_container_job

createTime: '2021-08-26T13:17:21Z'
etag: HfD_hO217bU=
jobId: Iris_data_custom_container_job
state: PREPARING
trainingInput:
  jobDir: gs://hassan-ai-platform-temp/iris-job-custom-container-dir
  masterConfig:
    imageUri: gcr.io/qp-acc-aia-migrate-2021-06/ai_platform_training_custom_container_image:latest
  region: us-west1
trainingOutput: {}

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/Iris_data_custom_container_job?project=qp-acc-aia-migrate-2021-06

View logs at:
https://console.cloud.google.com/logs?resource=ml_job%2Fjob_id%2FIris_data_custom_container_job&project=qp-acc-aia-migrate-2021-06


In [29]:
MODEL_NAME = "iris_model"
REGION = 'us-central1'
! gcloud ai-platform models create $MODEL_NAME \
  --regions $REGION


Using endpoint [https://ml.googleapis.com/]
Created ai platform model [projects/qp-acc-aia-migrate-2021-06/models/iris_model].


In [30]:
print(MODEL_NAME)

iris_model


In [22]:
BUCKET_NAME = 'hassan-ai-platform-temp'
MODEL_VERSION = "container_model_v1"
REGION ='us-central1'
JOB_DIR = 'gs://' + BUCKET_NAME
# Get a list of directories in the `keras_export` parent directory
KERAS_EXPORT_DIRS = ! gsutil ls $JOB_DIR
print(KERAS_EXPORT_DIRS[2])


gs://hassan-ai-platform-temp/model.joblib


In [23]:


# Pick the directory with the latest timestamp, in case you've trained
# multiple times
SAVED_MODEL_PATH = KERAS_EXPORT_DIRS[0]

# Create model version based on that SavedModel directory
! gcloud ai-platform versions create $MODEL_VERSION \
  --model $MODEL_NAME \
  --runtime-version 1.15 \
  --python-version 3.7 \
  --framework tensorflow \
  --region $REGION \
  --origin $SAVED_MODEL_PATH

Using endpoint [https://us-central1-ml.googleapis.com/]
ERROR: (gcloud.ai-platform.versions.create) NOT_FOUND: Field: parent Error: The model resource: "[iris_model]" was not found. Please create the model resource first by using 'gcloud ai-platform models create [iris_model]'.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: "The model resource: \"[iris_model]\" was not found. Please create\
      \ the model resource first by using 'gcloud ai-platform models create [iris_model]'."
    field: parent


In [40]:
MODEL_DIR="gs://hassan-ai-platform-temp/"
VERSION_NAME="[container_model_v1]"
MODEL_NAME="[iris_model]"
FRAMEWORK="[scikit-learn]"
REGION = 'us-central1'
CUSTOM_CODE_PATH="gs://hassan-ai-platform-temp/custom-container-job-dir/packages/1d4998efa973ab08e9ad7150a514e07355ec17232eedba5dc4770284d32a762a/trainer-0.0.0.tar.gz"
PREDICTOR_CLASS="[train].[CLASS_NAME]"

In [42]:
!gcloud beta ai-platform versions create $VERSION_NAME \
  --model=$MODEL_NAME \
  --origin=$MODEL_DIR \
  --runtime-version=2.5 \
  --python-version=3.7 \
  --region=$REGION \
  --package-uris=$CUSTOM_CODE_PATH

Using endpoint [https://us-central1-ml.googleapis.com/]
ERROR: (gcloud.beta.ai-platform.versions.create) NOT_FOUND: Field: parent Error: The model resource: "[iris_model]" was not found. Please create the model resource first by using 'gcloud ai-platform models create [iris_model]'.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: "The model resource: \"[iris_model]\" was not found. Please create\
      \ the model resource first by using 'gcloud ai-platform models create [iris_model]'."
    field: parent


## Prediction

In [34]:
%pwd
%cd ai_platform_training_custom_container/

/home/jupyter/ai_platform_training_custom_container


In [26]:
# !gsutil cp ./Iris_test.csv gs://$BUCKET_NAME
import pandas as pd
prediction_input = pd.read_csv('./iris.csv')

In [27]:
prediction_input1

,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa
...,...,...,...,...,...
144,6.7,3.0,5.2,2.3,Iris-virginica
145,6.3,2.5,5.0,1.9,Iris-virginica
146,6.5,3.0,5.2,2.0,Iris-virginica
147,6.2,3.4,5.4,2.3,Iris-virginica


In [28]:
prediction_input = prediction_input.iloc[:, :-1]

In [29]:
prediction_input

,5.1,3.5,1.4,0.2
0,4.9,3.0,1.4,0.2
1,4.7,3.2,1.3,0.2
2,4.6,3.1,1.5,0.2
3,5.0,3.6,1.4,0.2
4,5.4,3.9,1.7,0.4
...,...,...,...,...
144,6.7,3.0,5.2,2.3
145,6.3,2.5,5.0,1.9
146,6.5,3.0,5.2,2.0
147,6.2,3.4,5.4,2.3


In [30]:
import json

with open('prediction_input_container.json', 'w') as json_file:
  for row in prediction_input.values.tolist():
    json.dump(row, json_file)
    json_file.write('\n')

! cat prediction_input.json